In [2]:
import json
import bz2
import sys
import os
import pandas as pd
import time

In [14]:
WIKI_DATA_FILTERED = 'filtered_politician.json.bz2'
QUOTEBANK_POLITICIANS_WORLD= 'quotebank_politicians_world.json.bz2'
QUOTEBANK_POLITICIANS_USA= 'quotebank_politicians_USA.json.bz2'
quotebank_dir ='../data/'
print(type(os.listdir(quotebank_dir)))
quotebank_files=list( os.path.join(quotebank_dir,x) for x in os.listdir(quotebank_dir))
print(quotebank_files)
qids_df=pd.read_json(WIKI_DATA_FILTERED, lines=True)

<class 'list'>
['../data/quotes-2015.json-003.bz2', '../data/quotes-2016.json-002.bz2', '../data/quotes-2018.json-006.bz2', '../data/quotes-2019.json-004.bz2']


In [42]:
os.remove(QUOTEBANK_POLITICIANS_WORLD)
os.remove(QUOTEBANK_POLITICIANS_USA)
sum_quotes_world, total_time, sum_quotes_usa=0,0,0



with bz2.open(QUOTEBANK_POLITICIANS_WORLD, 'wb') as bzout_world:
    with bz2.open(QUOTEBANK_POLITICIANS_USA, 'wb') as bzout_usa:
        t1=time.time()
        quote_world=[]
        quote_usa=[]
        for file in quotebank_files:
            with bz2.open(file,'rt',encoding="utf8") as bzin:
                for i, item in enumerate(bzin):
                    
                    quote=json.loads(item)
                    qids=quote.get('qids')
                    if len(qids) != 0:
                        for quote_qid in qids:
                            if quote_qid in list(qids_df.qid):
                                quote_world.append(quote)
                                if qids_df.loc[qids_df.qid == quote_qid,'nationality'].values[0] == 'Q30':
                                    quote_usa.append(quote)
                                break #break as soon as one qid in is WIKI_DATA_FILTERED
                    if i % 1000 == 0:
                        bzout_world.write(('\n'.join(map(json.dumps, quote_world))+'\n').encode('utf-8'))
                        bzout_usa.write(('\n'.join(map(json.dumps, quote_usa))+'\n').encode('utf-8'))
                        t2=time.time()
                        dt=t2-t1
                        total_time += dt
                        print(f'dumped {len(quote_world)} WORLD and {len(quote_usa)} USA among {i} quotes in {dt}s')
                        t1=time.time()
                        sum_quotes_world += len(quote_world)
                        sum_quotes_usa += len(quote_usa)
                        quote_world=[]
                        quote_usa=[]
    print(f'WORLD: total quotes: {sum_quotes_world} in {total_time}s')
    print(f'USA: total quotes: {sum_quotes_usa} in {total_time}s')

dumped 1 WORLD and 1 USA among 0 quotes in 0.09316515922546387s
dumped 175 WORLD and 86 USA among 1000 quotes in 55.29328274726868s
dumped 155 WORLD and 85 USA among 2000 quotes in 56.82584857940674s
dumped 158 WORLD and 75 USA among 3000 quotes in 60.20851182937622s
dumped 146 WORLD and 68 USA among 4000 quotes in 56.64293050765991s
dumped 155 WORLD and 73 USA among 5000 quotes in 68.43807053565979s
dumped 147 WORLD and 83 USA among 6000 quotes in 60.72544527053833s
dumped 146 WORLD and 73 USA among 7000 quotes in 61.078824043273926s


KeyboardInterrupt: 

In [45]:
sumquote=0
with bz2.open(QUOTEBANK_POLITICIANS_USA, 'rt', encoding='utf8') as bzin_usa:
    for i, item in enumerate(bzin_usa):
        sumquote+=1
        if i%200 == 0: print(item)
    print(sumquote)

{"quoteID": "2015-08-31-000271", "quotation": "... a great day for veterans here in Littleton and all across the North Country. The men and women of our armed forces never fail our nation on the battlefield, and we must not fail them when they return home and enter civilian life.", "speaker": "Jeanne Shaheen", "qids": ["Q270316"], "date": "2015-08-31 02:10:00", "numOccurrences": 2, "probas": [["Jeanne Shaheen", "0.742"], ["None", "0.2359"], ["Kelly Ayotte", "0.0221"]], "urls": ["http://www.unionleader.com/article/20150831/NEWS12/150839902&source=RSS", "http://unionleader.com/article/20150831/NEWS12/150839902"], "phase": "E"}

{"quoteID": "2015-10-20-018726", "quotation": "confident that the courts ultimately will dismiss this taxpayer-funded political stunt, which would make healthcare more expensive for millions of Americans.", "speaker": "Katie Hill", "qids": ["Q58416634", "Q6377460"], "date": "2015-10-20 19:15:47", "numOccurrences": 2, "probas": [["Katie Hill", "0.9393"], ["None", "